# Assignment 2 - Epipolar Geometry and 3D Reconstruction 

First name:
<br>
Last name: 
<br>
Matriculation number:

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os

import numpy as np
from PIL import Image
import ipyvolume as ipv
import matplotlib.pyplot as plt

from utils import decompose_essential_matrix, infer_3d, ransac

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


AttributeError: module 'collections' has no attribute 'Mapping'

## Part II: 3D Model Reconstruction

### Load matched points
We provide a synthetic pair of images where noisy correspondences are known.

In [ ]:
left = np.array(Image.open(os.path.join('MatchedPoints','left.jpg')), dtype=np.float32).mean(2)/255
right = np.array(Image.open(os.path.join('MatchedPoints','right.jpg')), dtype=np.float32).mean(2)/255

In [ ]:
plt.subplot(1,2,1)
plt.imshow(left, cmap='gray')
plt.title('Left image')
plt.subplot(1,2,2)
plt.imshow(right, cmap='gray')
plt.title('Right image')

In [ ]:
ilias_username = None # TODO
A = np.loadtxt(f'MatchedPoints/Matched_Points_{ilias_username}.txt')

In [ ]:
M, N = A.shape
leftPoints = np.concatenate((A[:,2:4].T, np.ones((1, M))), axis=0)
rightPoints = np.concatenate((A[:,0:2].T, np.ones((1, M))), axis=0)

### Calibration matrix and focal length from the given file

In [ ]:
fl = 4
K = np.array([
    [-83.33333, 0.00000, 250.00000],
    [0.00000, -83.33333, 250.00000],
    [0.00000, 0.00000,   1.00000],
])

I = K.copy()

I[0,0] *= fl
I[1,1] *= fl

### Estimate Essential matrix E from F with RANSAC

The 8-point algorithm assumes that all entries are independent from one another.
Matrix E has 9 unknown entries but one can be derived from the other 8.
For 'n' points is possible to write an homogeneous system of n equations 
--> in this case to derive non trivial solutions for QE=0 -->(x'^T) * E * x=0 (Q same role as A for F)
I need at least 8 point correspondences, each one providing one independent equation
--> If I know the calibration matrices of the two cameras, I can estimate the essential matrix: E = K2^T*F*K1, inverse approach where:
- K1, internal camera calibration matrix of camera 1 --> x1_n=K^-1 * x1
- K2, internal camera calibration matrix of camera 2


In [ ]:
good_threshold = 1e-3 # or 0.1 # TODO FOR EXAMPLE
F, inliers = ransac(leftPoints, rightPoints, good_threshold)  
print('Num outliers', leftPoints.shape[1] - inliers.sum())
assert np.linalg.matrix_rank(F) == 2
print('Estimated fundamental matrix: ')
print(F)

# TODO: Estimate essential matrix E from F
#K1 and K2 are the intrinsics of the two cameras
#intrinsics = np.load('Intrinsic/and/extrinsic/parameters.rtf')#('../data/intrinsics.npz')
#K1 = intrinsics['K1']
#K2 = intrinsics['K2']
K1 = x1_n^(-1) *x1
K2 = x2_n^(-1) *x2

E = get_essential_matrix(F, K1, K2)

print('Estimated essential matrix: ')
print(E)

### Compute rotation and translation between views

In [ ]:
# TODO: Compute rotation and translation between views. Complete decomposeE
Il = np.linalg.solve(I, leftPoints)
Ir = np.linalg.solve(I, rightPoints)

Pl, Pr = decompose_essential_matrix(E, Il, Ir)

print('Estimated translation: ')
print(Pr[:,3])
print('Estimated rotation: ')
print(Pr[:,:3])

### Estimate the 3D points

In [ ]:
x3D = infer_3d(Il, Ir, Pl, Pr)
ipv.quickscatter(x=x3D[0,:], y=x3D[1,:], z=x3D[2,:])